**_pySpark Basics: Linear Regression_**

_By: Jeff Levy (jlevy@urban.org_

_Last Updated: 24 June 2016, Spark v1.6.1_

_Abstract: This guide will go over the basics of linear regression modeling using SGD in pySpark, and compare it to OLS._

***

We'll do this in four steps:

1. To begin with, we'll generate some sample data from a simple process so we know what the coefficients _should_ be
2. Then we'll analyze it with traditional OLS using a pseudoinverse
3. Then we'll analyze it with pySpark's method using stochastic gradient descent 
4. And finally we'll compare the known population paramaters and the two different forms of estimating them

First we have to install a few Python modules:

In [3]:
import pip
#pip.main(['install', 'pandas==0.16.2'])
pip.main(['install', '--upgrade', 'pandas'])
#pip.main(['install', 'statsmodels'])

  Using cached pandas-0.18.1.tar.gz
Requirement already up-to-date: python-dateutil in ./venv/lib/python2.7/site-packages (from pandas)
Requirement already up-to-date: pytz>=2011k in ./venv/lib/python2.7/site-packages (from pandas)
Requirement already up-to-date: numpy>=1.7.0 in ./venv/lib64/python2.7/site-packages (from pandas)
Requirement already up-to-date: six>=1.5 in ./venv/lib/python2.7/site-packages (from python-dateutil->pandas)
  Found existing installation: pandas 0.16.2
    Uninstalling pandas-0.16.2:
      Successfully uninstalled pandas-0.16.2
  Running setup.py install for pandas: started
    Running setup.py install for pandas: still running...
    Running setup.py install for pandas: still running...
    Running setup.py install for pandas: finished with status 'done'


0

In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

In [2]:
pd.__version__

u'0.18.1'

Now we'll build a dataframe using Pandas.  This is the module used for non-distributed data analysis in Python - it cannot be used for the sort of big data Spark is designed for.  However, since we're dealing with toy data we can use it here just fine, and it will work in the memory of a single machine.

In [3]:
data = {'y':[], 'x1':[], 'x2':[], 'x3':[]}
a, b1, b2, b3 = 1, .2, .4, .5     #population paramters: intercept (a) and betas (b1, b2, b3)

def make_row(data):
    #generating some random x values for our data:
    x1 = np.random.normal(10,2)
    x2 = np.random.normal(17,3)
    x3 = np.random.normal(8,1)
    e = np.random.normal(0, .1) #a small error term
    
    #calculating our dependent variable from the data and the known population paramters
    y = a + b1*x1 + b2*x2 + b3*x3 + e
    
    #adding the results for this observation to the data
    data['x1'].append(x1)
    data['x2'].append(x2)
    data['x3'].append(x3)
    data['y'].append(y)
    return data
    
#creating 200 observations
for _ in range(200):
    data = make_row(data)

Now we load it into a Pandas dataframe and look at the first five rows:

In [4]:
pdf = pd.DataFrame(data)
pdf[:5]

,x1,x2,x3,y
0,10.144736,18.529474,9.760265,15.299948
1,12.301140,15.440311,8.625533,13.918040
2,8.503888,18.697738,7.518186,13.925267
3,10.164505,15.225154,9.609322,13.670369
4,9.388564,16.854149,8.004604,13.821654


Now we use statsmodels, a Python module for statistical tools (again, not for distributed work), to do a normal OLS involving a pseudoinverse:

In [8]:
model = sm.OLS(pdf['y'], sm.add_constant(pdf[['x1', 'x2', 'x3']]))
results = model.fit()

In [9]:
results.params

const    1.018746
x1       0.202988
x2       0.399515
x3       0.494584
dtype: float64

In [10]:
results.rsquared

0.99513368576594396

In [11]:
results.pvalues

const     4.633138e-28
x1       1.021059e-125
x2       1.367133e-220
x3       1.324201e-138
dtype: float64

And we essentially get our original paramters back, have an R^2 of nearly one, and highly significant pvalues ust as we would expect (with some variation because we added an error term).  

Now let's try the same thing using the tools avilable in pySpark that will work on very large distributed data.  It can build the RDD-based dataframe that we need straight from the Pandas dataframe:

In [6]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [5]:
sdf = sqlContext.createDataFrame(pdf)

In [6]:
sdf.show()

+------------------+------------------+------------------+------------------+
|                x1|                x2|                x3|                 y|
+------------------+------------------+------------------+------------------+
|10.144736104845618|18.529474419107736| 9.760264704880655|15.299947600283975|
|  12.3011404989677| 15.44031056813525| 8.625532921681394| 13.91803995704791|
| 8.503887608599818|18.697738478838776|7.5181857229865185|13.925266971339544|
|10.164505228273514|15.225154016677319| 9.609321544034191|13.670368953178047|
| 9.388563702719713|16.854148559954158| 8.004604345737487|13.821654454602573|
|10.516058712914345|15.437628379951642|6.6957112434300345|12.530089286713215|
|12.660066405263873|14.869981914637247| 7.783816162982566|13.290384811728076|
| 9.930156485396305|15.785596093138121| 8.029172107453952|13.296763429620276|
| 10.53923972522434| 19.92897605777253| 8.329562697075811|15.184952697233019|
| 9.376886357328205| 16.90436494733737| 9.457475498602589|14.475